In [1]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData

stockdata = GetStockData() 
spark = DevSparkSession().spark


ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d965c7d3-7092-430e-97ef-c50cd8365a28;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/1.2.1/delta-core_2.12-1.2.1.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;1.2.1!delta-core_2.12.jar (433ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/1.2.1/delta-storage-1.2.1.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;1.2.1!delta-storage.jar (89ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (93ms)


In [2]:
from pyspark.sql.functions import col,explode,sha2,concat_ws,trim, lit

In [3]:
json_data  = stockdata.get_historical_stock_data()


data saved for symbol QQQ and date range 2021-01-01 to 2021-12-31.
data saved for symbol VOO and date range 2021-01-01 to 2021-12-31.


In [4]:
print(json_data[0].get("meta", {}).get("symbol"))

QQQ


In [5]:

df = spark.createDataFrame(json_data) 

df = df.withColumn("symbol", col("meta").symbol).filter( col("status") == "ok" ).drop(df.status)

df.show(truncate = False)




+--------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:

df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy( "symbol" ) \
    .save("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze")


In [ ]:
audit_columns =   [
    "__ActivationDateTime",
    "__BusinessKeyHash",
    "__CreateDateTime",
    "__CreatedBatchLogId",
    "__CurrentFlag",
    "__DeactivationDateTime",
    "__DeletedFlag",
    "__EffectiveEndDateTime",
    "__EffectiveStartDateTime",
    "__FactKeyHash",
    "__Hash1Type",
    "__Hash2Type",
    "__HashKey",
    "__HashValue",
    "__LastModified",
    "__UpdateDateTime",
    "__UpdatedBatchLogId"
] 

audit_columns_upper =  {column for  column  in audit_columns }


FileNotFoundError: [Errno 2] No such file or directory: '/private/var/folders/_v/nhdy5ft93pxfgvlcsxxtdt5c0000gn/T/spark-38108acf-ca25-4320-a03a-efc4957216f1/userFiles-7e5bd364-5726-48c9-a0cf-2b36dc474e79/io.delta_delta-core_2.12-1.2.1.jar'

In [12]:
from pyspark.sql import SparkSession

active_session = SparkSession.getActiveSession()

active_session

In [ ]:
import pwhi

3.5.3


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, sha2, concat_ws

# 1. Create SparkSession
spark = SparkSession.builder.appName("DeltaMergeInitialLoad").getOrCreate()

# Paths for Bronze and Silver data
bronze_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"  # Bronze data stored in Parquet or Delta
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver"  # Target location for Silver Delta table

# 2. Load Bronze Data
df_bronze = spark.read.parquet(bronze_path)

# 3. Create the Silver Delta Table if it doesn't exist
if not DeltaTable.isDeltaTable(spark, silver_path):
    # Option A: Create the Silver table using the Bronze schema (with no rows)
    # It's common to add audit columns here if they are not present in Bronze.
    df_empty = df_bronze.limit(0)
    
    # Alternatively, you may want to explicitly define the schema for Silver including audit columns.
    df_empty.write.format("delta").mode("overwrite").save(silver_path)
    print("Created empty Silver Delta table.")

# Load the Silver table as a DeltaTable object
silverDeltaTable = DeltaTable.forPath(spark, silver_path)

type(silverDeltaTable)

# # 4. Perform the Delta Lake merge
# # Define the merge condition based on your business key(s) (here "business_key")
# # You can expand the merge logic as needed for your schema.
# silverDeltaTable.alias("t").merge(
#     source = df_bronze.alias("s"),
#     condition = "t.business_key = s.business_key"  # Adjust the key condition as needed
# ).whenMatchedUpdate(
#     set = {
#         "some_column": "s.some_column",
#         "__CurrentFlag": "true",
#         "__DeletedFlag": "false",
#         # Updating the old record with a deactivation timestamp, if applicable
#         "__DeactivationDateTime": "current_timestamp()",
#         "__lastmodified": "current_timestamp()",
#         "__HashKey": "sha2(concat_ws('|', s.business_key, s.some_column), 256)",
#         "__HashValue": "sha2(concat_ws('|', s.business_key, s.some_column), 256)"
#     }
# ).whenNotMatchedInsert(
#     values = {
#         "business_key": "s.business_key",
#         "some_column": "s.some_column",
#         "__CurrentFlag": "true",
#         "__DeletedFlag": "false",
#         "__ActivationDateTime": "current_timestamp()",
#         "__DeactivationDateTime": None,
#         "__lastmodified": "current_timestamp()",
#         "__HashKey": "sha2(concat_ws('|', s.business_key, s.some_column), 256)",
#         "__HashValue": "sha2(concat_ws('|', s.business_key, s.some_column), 256)"
#     }
# ).execute()

# print("Merge operation complete.")


AttributeError: module 'pyspark.sql.utils' has no attribute 'convert_exception'

In [17]:
TypeIColumns = ""
TypeIColumnsList   = [col.strip()  for col in TypeIColumns.split(",") if  col !=  ""] 

# TypeIColumnsList = None if TypeIColumnsList = []

 
print( TypeIColumnsList )

[]


In [24]:
from delta.tables import DeltaTable

df_raw = spark.read.load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"); 

df_raw.show()



+--------------------+--------------------+------+
|                meta|              values|symbol|
+--------------------+--------------------+------+
|{exchange_timezon...|[{volume -> 23850...|   QQQ|
|{exchange_timezon...|[{volume -> 37518...|   VOO|
+--------------------+--------------------+------+



In [ ]:

from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType

schema_meta = StructType([
                    StructField('exchange_timezone',StringType(), False),
                    StructField('symbol',StringType(), False),
                    StructField('mic_code', StringType(), False),
                    StructField('NASDAQ', StringType(), False),
                    StructField('currency',StringType(), False),
                    StructField('type',StringType(), False),
                    StructField('interval',StringType(), False)
                     
            ])


df_silver = spark.createDataFrame([]   df_raw.select("meta") )


PySparkTypeError: [SHOULD_NOT_DATAFRAME] Argument `data` should not be a DataFrame.

In [ ]:
df = spark.read.load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"); 

df_dim = df.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimezone")
    ) 

businessColumns = "Symbol,ExchangeName,Currency" 
businessColumnsList   = [col.strip()  for col in businessColumns.split(",") if  col !=  ""]


TypeIColumns = "Symbol"
TypeIColumnsList   = [col.strip()  for col in TypeIColumns.split(",") if  col !=  ""]



TypeIIColumns = ""
TypeIColumnsList   =  list({col for col in  df_dim.columns}  - audit_columns_upper  -  set(TypeIColumnsList))


# df_dim = df_dim.withColumn("__HashKey", sha2( col("Symbol"), 256)) 
# df_dim = df_dim.withColumn("__HashKey", sha2( concat_ws("|",  *businessColumnsList), 256)) 

 
# profile data
df_dim.limit(1).show(truncate = False) 


TypeIColumnsList

 

+------+------------+--------+----+----------------+
|Symbol|ExchangeName|Currency|Type|ExchangeTimezone|
+------+------------+--------+----+----------------+
|QQQ   |NASDAQ      |USD     |ETF |America/New_York|
+------+------------+--------+----+----------------+



['Currency', 'Type', 'ExchangeTimezone', 'ExchangeName']

In [ ]:

__Current
__Deleted
__ActivationDateTime
__DeactivationDateTime
__HashKey
__HashValue
__lastmodified




In [ ]:

df_flat_exploded = df_flat2.select(
                        col("symbol"),
                        explode( "values").alias("record"))
                        
                        # .withColumn( "volume" , col("record").getItem("volume") ) \
                        # .withColumn( "high" , col("record").getItem("high") ) \
                        # .withColumn( "low" , col("record").getItem("low") ) \
                        # .withColumn( "close" , col("record").getItem("close") ) \
                        # .withColumn( "open" , col("record").getItem("open") ) \
                        # .withColumn( "date" , col("record").getItem("datetime") ) \
                        #         .drop("record")


df_flat_exploded =   df_flat_exploded.select(
        "symbol",
        col("record").getItem("volume").alias("volume"),
        col("record").getItem("high").alias("high"),
        col("record").getItem("low").alias("low"),
        col("record").getItem("close").alias("close"),
        col("record").getItem("open").alias("open"),
        col("record").getItem("date").alias("date")
        
  )   
df_flat_exploded = df_flat_exploded.filter(  col("symbol")  == "VOO" ).distinct().count()     


print(df_flat_exploded)
# df_flat_exploded.show(truncate = False )

251
